In [ ]:
import requests
import pandas as pd
from tqdm import tqdm
import time
from datetime import datetime, timedelta
from dateutil.relativedelta import relativedelta
from concurrent.futures import ThreadPoolExecutor, as_completed
from requests.adapters import HTTPAdapter
from urllib3.util.retry import Retry

HEADERS = {"accept": "application/json"}

# 연결 풀을 재사용하는 세션 생성
def create_session():
    """재시도 로직과 연결 풀이 있는 세션 생성"""
    session = requests.Session()
    retry = Retry(
        total=3,
        backoff_factor=0.3,
        status_forcelist=[500, 502, 503, 504]
    )
    adapter = HTTPAdapter(
        max_retries=retry,
        pool_connections=50,
        pool_maxsize=50
    )
    session.mount("http://", adapter)
    session.mount("https://", adapter)
    return session

# 전역 세션
session = create_session()


# ----------------------------------------------------------
# 헬퍼 함수
# ----------------------------------------------------------
def simple_list_to_str(lst, key="name"):
    return ", ".join([item.get(key, "") for item in lst]) if lst else ""


def generate_date_periods(start_date, end_date, months=3):
    """
    시작일과 종료일 사이를 N개월 단위로 분할 (정확한 달 계산)
    """
    from dateutil.relativedelta import relativedelta
    
    periods = []
    current = datetime.strptime(start_date, "%Y-%m-%d")
    end_dt = datetime.strptime(end_date, "%Y-%m-%d")
    
    while current <= end_dt:
        # 다음 기간의 시작일 계산 (정확히 N개월 후)
        next_date = current + relativedelta(months=months)
        period_end = min(next_date - timedelta(days=1), end_dt)
        
        periods.append((
            current.strftime("%Y-%m-%d"),
            period_end.strftime("%Y-%m-%d")
        ))
        
        current = next_date
    
    return periods


# ----------------------------------------------------------
# 1) 지정된 기간 동안 방영된 TV Series 전체 수집 (멀티스레드)
# ----------------------------------------------------------
def fetch_single_page(page, start_date, end_date):
    """단일 페이지를 가져오는 함수"""
    base_url = "https://api.themoviedb.org/3/discover/tv"
    params = {
        "api_key": API_KEY,
        "language": "en-US",
        "sort_by": "popularity.desc",
        "first_air_date.gte": start_date,
        "first_air_date.lte": end_date,
        "page": page,
    }
    
    try:
        response = session.get(base_url, params=params, headers=HEADERS, timeout=10)
        data = response.json()
        return data.get("results", []), data.get("total_pages", 1), data.get("total_results", 0)
    except Exception as e:
        print(f"페이지 {page}에서 오류: {e}")
        return [], 1, 0


def fetch_tv_series_between_dates(start_date, end_date):
    """
    TMDB Discover API를 사용하여 특정 기간 내 모든 TV Series 목록을 수집 (멀티스레드).
    최대 500페이지(10,000개) 제한 적용
    """
    # 먼저 첫 페이지를 가져와서 총 페이지 수 확인
    results, total_pages, total_results = fetch_single_page(1, start_date, end_date)
    all_series = results
    
    # 500페이지 제한 적용
    max_pages = min(total_pages, 500)
    
    print(f"  📊 총 {total_results:,}개 ({total_pages}페이지) → 수집 가능: {max_pages}페이지")
    
    if total_pages > 500:
        print(f"  ⚠️  500페이지 제한으로 인해 {(total_pages - 500) * 20:,}개는 수집 불가")
    
    if max_pages == 1:
        return all_series
    
    # 나머지 페이지를 멀티스레드로 수집
    with ThreadPoolExecutor(max_workers=20) as executor:
        futures = {
            executor.submit(fetch_single_page, page, start_date, end_date): page 
            for page in range(2, max_pages + 1)
        }
        
        for future in tqdm(as_completed(futures), total=len(futures), desc=f"  페이지 수집", leave=False):
            page_results, _, _ = future.result()
            all_series.extend(page_results)
    
    return all_series


# ----------------------------------------------------------
# 2) 리뷰 수집 (모든 페이지)
# ----------------------------------------------------------
def fetch_reviews(content_type, content_id):
    """
    모든 리뷰를 수집 (페이지네이션 처리)
    """
    url = f"https://api.themoviedb.org/3/{content_type}/{content_id}/reviews"
    all_reviews = []
    page = 1
    
    while True:
        params = {"api_key": API_KEY, "language": "en-US", "page": page}
        
        try:
            response = session.get(url, params=params, headers=HEADERS, timeout=10)
            if response.status_code != 200:
                break
            
            data = response.json()
            reviews = data.get("results", [])
            
            if not reviews:
                break
            
            for review in reviews:
                author = review.get("author", "unknown")
                rating = review.get("author_details", {}).get("rating")
                rating_str = f" (Rating: {rating})" if rating else ""
                content = review.get("content", "").replace("\n", " ").replace("\r", " ")
                all_reviews.append(f"{author}{rating_str}: {content}")
            
            # 다음 페이지가 있는지 확인
            total_pages = data.get("total_pages", 1)
            if page >= total_pages:
                break
            
            page += 1
            
        except:
            break
    
    return " || ".join(all_reviews) if all_reviews else ""


# ----------------------------------------------------------
# 3) TV Series 상세 정보 수집
# ----------------------------------------------------------
def fetch_tv_details(series_id):
    url = f"https://api.themoviedb.org/3/tv/{series_id}"
    params = {"api_key": API_KEY, "language": "en-US", "append_to_response": "credits"}

    try:
        response = session.get(url, params=params, headers=HEADERS, timeout=10)
        data = response.json()

        # last_episode_to_air 분리
        last_ep = data.get("last_episode_to_air", {}) or {}

        record = {
            "id": data.get("id"),
            "type": "tv_series",
            "adult": data.get("adult"),
            "backdrop_path": data.get("backdrop_path"),
            "created_by": simple_list_to_str(data.get("created_by", [])),
            "episode_run_time": ", ".join(map(str, data.get("episode_run_time", []))) if data.get("episode_run_time") else "",
            "first_air_date": data.get("first_air_date"),
            "genres": simple_list_to_str(data.get("genres", [])),
            "homepage": data.get("homepage"),
            "in_production": data.get("in_production"),
            "languages": ", ".join(data.get("languages", [])),
            "last_air_date": data.get("last_air_date"),

            # last_episode_to_air를 개별 컬럼으로 분리
            "last_episode_to_air_id": last_ep.get("id"),
            "last_episode_to_air_name": last_ep.get("name"),
            "last_episode_to_air_overview": last_ep.get("overview"),
            "last_episode_to_air_vote_average": last_ep.get("vote_average"),
            "last_episode_to_air_vote_count": last_ep.get("vote_count"),
            "last_episode_to_air_air_date": last_ep.get("air_date"),
            "last_episode_to_air_episode_number": last_ep.get("episode_number"),
            "last_episode_to_air_production_code": last_ep.get("production_code"),
            "last_episode_to_air_runtime": last_ep.get("runtime"),
            "last_episode_to_air_season_number": last_ep.get("season_number"),
            "last_episode_to_air_show_id": last_ep.get("show_id"),
            "last_episode_to_air_still_path": last_ep.get("still_path"),

            "name": data.get("name"),
            "next_episode_to_air": None,
            "networks": simple_list_to_str(data.get("networks", [])),
            "number_of_episodes": data.get("number_of_episodes"),
            "number_of_seasons": data.get("number_of_seasons"),
            "origin_country": ", ".join(data.get("origin_country", [])),
            "original_language": data.get("original_language"),
            "original_name": data.get("original_name"),
            "overview": data.get("overview"),
            "popularity": data.get("popularity"),
            "poster_path": data.get("poster_path"),
            "production_companies": simple_list_to_str(data.get("production_companies", [])),
            "production_countries": simple_list_to_str(data.get("production_countries", [])),
            "seasons": "; ".join(
                f"{s.get('season_number')}: {s.get('name')} ({s.get('episode_count')} eps)"
                for s in data.get("seasons", [])
            ),
            "spoken_languages": simple_list_to_str(data.get("spoken_languages", [])),
            "status": data.get("status"),
            "tagline": data.get("tagline"),
            "type_detail": data.get("type"),
            "vote_average": data.get("vote_average"),
            "vote_count": data.get("vote_count"),
        }

        record["review"] = fetch_reviews("tv", series_id)

        return record

    except Exception as e:
        print(f"Error fetching details for {series_id}: {e}")
        return None


# ----------------------------------------------------------
# 4) MAIN 실행부
# ----------------------------------------------------------
if __name__ == "__main__":
    print("=" * 90)
    print("🚀 PERIOD-SPLIT MULTITHREAD VERSION — TV SERIES (2023.01.01 ~ 2025.06.30)")
    print("=" * 90)

    start_date = "2023-01-01"
    end_date = "2025-06-30"

    total_start = time.time()

    # 기간을 3개월 단위로 분할
    periods = generate_date_periods(start_date, end_date, months=3)
    
    print(f"\n📅 전체 기간: {start_date} ~ {end_date}")
    print(f"📊 분할 기간: {len(periods)}개 (각 3개월씩)")
    print("="*90)

    all_tv_series = []

    # 각 기간별로 데이터 수집
    for idx, (period_start, period_end) in enumerate(periods, 1):
        print(f"\n[{idx}/{len(periods)}] 📌 기간: {period_start} ~ {period_end}")
        
        tv_series = fetch_tv_series_between_dates(period_start, period_end)
        all_tv_series.extend(tv_series)
        
        print(f"  ✅ 이 기간 수집: {len(tv_series):,}개 (누적: {len(all_tv_series):,}개)")

    if not all_tv_series:
        print("\n⚠️ 수집된 데이터 없음. 종료합니다.")
        exit()

    # 중복 제거
    print(f"\n{'='*90}")
    print(f"🧹 중복 제거 중...")
    unique_series = {s["id"]: s for s in all_tv_series}
    tv_list = list(unique_series.values())
    print(f"✨ 중복 제거 전: {len(all_tv_series):,}개 → 중복 제거 후: {len(tv_list):,}개")
    print(f"   (제거된 중복: {len(all_tv_series) - len(tv_list):,}개)")

    print(f"\n{'='*90}")
    print("📌 상세 정보 멀티스레드 수집 시작...\n")

    results = []
    with ThreadPoolExecutor(max_workers=30) as executor:
        futures = {executor.submit(fetch_tv_details, s["id"]): s for s in tv_list}

        for future in tqdm(as_completed(futures), total=len(futures), desc="TV 상세 수집"):
            detail = future.result()
            if detail:
                results.append(detail)

    print(f"\n🎉 상세 정보 수집 완료: {len(results):,}개")

    # 저장
    df = pd.DataFrame(results)
    df = df.sort_values(["first_air_date", "popularity"], ascending=[True, False])
    df.to_csv("tv_series_2023_2025_JanToJun_FULL.csv", index=False, encoding="utf-8-sig")

    elapsed = time.time() - total_start

    print("\n" + "="*90)
    print("========================== DONE ==========================")
    print(f"📌 총 데이터: {len(df):,}개")
    print(f"⏱  소요시간: {elapsed/60:.2f}분 ({elapsed:.2f}초)")
    print(f"💾 저장 파일: tv_series_2023_2025_JanToJun_FULL.csv")
    print(f"\n🚀 성능 개선 포인트:")
    print(f"   ✅ 기간 분할로 500페이지 제한 우회 (전체 데이터 수집)")
    print(f"   ✅ 페이지 수집 멀티스레드화 (20개 동시)")
    print(f"   ✅ 상세 정보 수집 스레드 증가 (30개 동시)")
    print(f"   ✅ HTTP 연결 풀 재사용 (TCP 연결 오버헤드 제거)")
    print(f"   ✅ 자동 재시도 로직 (네트워크 오류 자동 복구)")
    print("="*90)